# Analysis of Robot reports from OSM Jenkins

In [ ]:
import os
import pandas as pd
import numpy as np
import datetime as dt
#import getpass
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from IPython.display import display, Markdown

## Update database

In [ ]:
# Uncomment to update the database from this notebook
%run 00-script-jenkins_and_robot_etl.py

## 0. Input parameters

In [ ]:
inputs_folder = 'etl_outputs'
outputs_folder = 'report_outputs'
database_uri = f'sqlite:///{inputs_folder}/test_executions.db'

too_old_builds = "2020-12-15"

# Comment for analysis of all historical data
days_since_today_4_analysis = 21

extended_print = False
dump_sequences = True

In [ ]:
relevant_jobs = ['osm-stage_3-merge/v10.0', 'osm-stage_3-merge/master', 'osm-stage_3-merge/v9.0']
job_names = ['Release TEN', 'Master branch', 'Release NINE']

In [ ]:
today_as_datetime = pd.to_datetime("today")
today = today_as_datetime.strftime('%Y-%m-%d')

display(Markdown(f'**Date and time of the report:** {today_as_datetime}'))

In [ ]:
if 'days_since_today_4_analysis' in locals():
    first_date_as_datetime = today_as_datetime - dt.timedelta(days=days_since_today_4_analysis)
    first_date = first_date_as_datetime.strftime('%Y-%m-%d')
else:
    first_date = too_old_builds # Unconstrained

last_date = today

In [ ]:
# Uncomment to set the window of analysis manually
#
# first_date = "2021-08-01"
# last_date = "2021-08-20"

In [ ]:
display(Markdown(f'**Analysed period:** {first_date} to {last_date}.'))

## 1. Retrieval of all currrent data for aggregate analytics

In [ ]:
def load_known_builds(engine, too_old_builds='1980-12-15'):

    table_known_builds = 'builds_info'

    query_known_builds = f'SELECT * FROM {table_known_builds} WHERE timestamp>DATETIME("{too_old_builds}")'

    with engine.begin() as conn:
        df_known_builds = pd.read_sql(query_known_builds, con=conn)

    # Fixes some special data types
    df_known_builds['timestamp'] = pd.to_datetime(df_known_builds.timestamp)
    df_known_builds['job'] = df_known_builds.job.astype('category')
    df_known_builds['duration'] = pd.to_timedelta(df_known_builds.duration.astype('float')*1000, unit='us')
    df_known_builds['build_result'] = df_known_builds.build_result.astype('category')
    df_known_builds['test_result'] = df_known_builds.test_result.astype('category')

    return df_known_builds

In [ ]:
def load_all_build_reports(engine, too_old_builds='1980-12-15'):

    table_known_builds = 'builds_info'
    table_robot_reports = 'robot_reports'

    query_robot_reports = f'''
    SELECT main.timestamp, details.*
    FROM {table_robot_reports} AS details
    INNER JOIN {table_known_builds} AS main
    ON details.job=main.job AND details.build=main.build
    WHERE main.timestamp>DATETIME("{too_old_builds}")
    ORDER BY details.job, details.build, details.starttime
    '''

    with engine.begin() as conn:
        df_all_build_reports = pd.read_sql(query_robot_reports, con=conn)

    # Fixes some special data types
    df_all_build_reports['timestamp'] = pd.to_datetime(df_all_build_reports.timestamp)
    df_all_build_reports['job'] = df_all_build_reports.job.astype('category')
    df_all_build_reports['id'] = df_all_build_reports.id.astype('category')
    df_all_build_reports['name'] = df_all_build_reports.name.astype('category')
    df_all_build_reports['source'] = df_all_build_reports.source.astype('category')
    df_all_build_reports['starttime'] = pd.to_datetime(df_all_build_reports.starttime)
    df_all_build_reports['endtime'] = pd.to_datetime(df_all_build_reports.endtime)
    df_all_build_reports['status'] = df_all_build_reports.status.astype('category')
    df_all_build_reports['failed_test_id'] = df_all_build_reports.failed_test_id.astype('category')
    df_all_build_reports['failed_test_name'] = df_all_build_reports.failed_test_name.astype('category')
    df_all_build_reports['failed_keyword'] = df_all_build_reports.failed_keyword.astype('category')

    return df_all_build_reports

In [ ]:
def load_all_build_reports_details(engine, too_old_builds='1980-12-15'):

    query_robot_reports_extended = f'''
    SELECT main.timestamp, details.*
    FROM {table_robot_reports_extended} AS details
    INNER JOIN {table_known_builds} AS main
    ON details.job=main.job AND details.build=main.build
    WHERE main.timestamp>DATETIME("{too_old_builds}")
    ORDER BY details.job, details.build, details.starttime
    '''

    with engine.begin() as conn:
        df_all_build_reports_details = pd.read_sql(query_robot_reports_extended, con=conn)

    # Fixes some special data types
    df_all_build_reports_details['timestamp'] = pd.to_datetime(df_all_build_reports_details.timestamp)
    df_all_build_reports_details['job'] = df_all_build_reports_details.job.astype('category')
    df_all_build_reports_details['suite_id'] = df_all_build_reports_details.suite_id.astype('category')
    df_all_build_reports_details['suite_name'] = df_all_build_reports_details.suite_name.astype('category')
    df_all_build_reports_details['test_id'] = df_all_build_reports_details.test_id.astype('category')
    df_all_build_reports_details['test_name'] = df_all_build_reports_details.test_name.astype('category')
    df_all_build_reports_details['keyword_name'] = df_all_build_reports_details.keyword_name.astype('category')
    df_all_build_reports_details['starttime'] = pd.to_datetime(df_all_build_reports_details.starttime)
    df_all_build_reports_details['endtime'] = pd.to_datetime(df_all_build_reports_details.endtime)
    df_all_build_reports_details['status'] = df_all_build_reports_details.status.astype('category')

    return df_all_build_reports_details

In [ ]:
print('Retrieving from database...\t' , end='')

In [ ]:
engine = create_engine(database_uri)

df_known_builds = load_known_builds(engine, too_old_builds=too_old_builds)
df_all_build_reports = load_all_build_reports(engine, too_old_builds=too_old_builds)
df_all_build_reports_details = load_all_build_reports_details(engine, too_old_builds=too_old_builds)

In [ ]:
# Adds columns with % of passed/failed sub-tests
df_known_builds = (
    df_known_builds
    .copy()
    .assign(pass_pct = lambda x: x.pass_count / (x.pass_count + x.fail_count))
    .fillna({'pass_pct': 0})
    .assign(fail_pct = lambda x: 1- x.pass_pct)
    .fillna({'fail_pct': 100})
)

In [ ]:
# df_known_builds.query('test_result=="PASS"').groupby('job').last()
# df_all_build_reports.info()
# df_all_build_reports_details.info()

In [ ]:
print('DONE')

## 2. Aggregated analysis of stability

### 2.1 Restricts data to time window for analysis

In [ ]:
# Convenient alias to allow time masks

#data = df_known_builds.query('(timestamp>"2021-08-01") & (timestamp<"2021-08-20")').copy()
data = df_known_builds

if 'first_date' in locals():
    data = data.query('timestamp>=@first_date')

if 'last_date' in locals():
    # Needs to include latest hour of the last day
    last_timestamp = pd.Timestamp(last_date) + dt.timedelta(days=1)
    data = data.query('timestamp<@last_timestamp')

data = data.copy()

### 2.2 Finding sequences of successful builds and Robot reports

In [ ]:
# Aligns markdown tables to the left

In [ ]:
%%html
<style>
table {align:left;display:block}
</style>

Two values retrieved from the build are key to determine the state of the jubs and test suites at a given moment:

- `build_result` is the outcome of the build, reported by Jenkins. It can be: `SUCCESS`, `FAILURE`, `UNSTABLE` or `ABORTED`.
- `test_result` is the summary of the concerned Robot tests. It can be: `FAIL`, `UNAVAILABLE` or `PASS`.

Based on these two states, 3 types of temporal sequences of success/failure are identified per builds and test suites:

1. Successful builds/failed builds in a row: `grp_build_result`.
2. Successful test reports vs. test reports with fails in a row: `grp_test_result`.
3. Clean builds and tests vs. failures (of any kind) in a row: `grp_success_fail`.

For the identification of these sequences, the following mapping applies:

| Type of sequence   | Relevant state | OK sequence contains    | NOK sequence contains   | Ignore        |
|--------------------|----------------|-------------------------|-------------------------|---------------|
| `grp_build_result` | `build_result` | `SUCCESS` or `UNSTABLE` | `FAILURE`               | `ABORTED`     |
| `grp_test_result`  | `test_result`  | `PASS`                  | `FAIL`                  | `UNAVAILABLE` |
| `grp_success_fail` | `test_result`  | `PASS`                  | `FAIL` or `UNAVAILABLE` | N/A           |


The different groupings of segments are detected and a label is added to each sample...

In [ ]:
mapping_build_result = {
    'SUCCESS': True,
    'UNSTABLE': True,
    'FAILURE': False
    # 'ABORTED' will yield 'N/A'
}

mapping_test_result = {
    'PASS': True,
    'FAIL': False
    # 'UNAVAILABLE' will yield 'N/A'
}

mapping_success_fail = {
    'PASS': True,
    'FAIL': False,
    'UNAVAILABLE': False
}

In [ ]:
def find_sequence_number(df, relevant_col, mapping, grouping=['job']):

    there_is_change = lambda x: x != x.shift()

    return df.groupby(grouping)[relevant_col].transform(
        lambda x: (
            x
            .map(mapping)
            .fillna(method='ffill')
            .fillna(method='bfill')  # Extrapolation if first samples are inconclusive (i.e. should be ignored)
            .pipe(there_is_change)
            .cumsum()
            .astype(int)
        )
    )

In [ ]:
# Adds columns with groups labels
data = (
    data
    .assign(
        grp_build_result = lambda x: find_sequence_number(x, relevant_col='build_result', mapping=mapping_build_result),
        grp_test_result = lambda x: find_sequence_number(x, relevant_col='test_result', mapping=mapping_test_result),
        grp_success_fail = lambda x: find_sequence_number(x, relevant_col='test_result', mapping=mapping_success_fail)
        )
)

In [ ]:
print('DONE')

Then, the duration of each of the sequences of success/failure is determined and a specific dataframe is built summarizing such sequences, to ease their representation and analysis...

In [ ]:
# Utility functions to extract sequences from a dataframe with test samples

def show_me(df):
    display(df)
    return df

def flatten_multi_level(df, outcome_name):

    temp_name = list(df.columns.values)[-1][0]
    df.columns = ['_'.join((col[1], col[0])) if (col[1] and col[0]!=temp_name) else col[0] for col in df.columns.values]
    df.rename(columns={temp_name: outcome_name}, inplace=True)

    return df

def extend_sequence(df, agg):
    df = df.copy()

    left_shifted = df.groupby(agg).min_timestamp.shift(-1)
    not_null = ~ left_shifted.isna()

    df.loc[not_null, 'max_timestamp'] = left_shifted.loc[not_null]

    return df

def extend_last_sample_per_group(df, agg):
    df = df.copy()

    max_right_edge = max(df.min_timestamp.max(), df.max_timestamp.max())

    last_item_indexes = df.groupby(agg).tail(1).index
    df.loc[last_item_indexes, 'max_timestamp'] = max_right_edge

    return df

def extend_lastest_build_per_job(df):
    df = df.copy()

    # Finds indices of rows generated from latest build of each job
    indices_latest_build_per_job = (
        df.groupby('job')
        .max_build
        .transform(lambda col: (col==col.max()))
    )

    # In those samples, changes 'max_timestamp' to the maximum of:
    # - Current max_timestamp + 12 hours
    # - Now
    df.loc[indices_latest_build_per_job, 'max_timestamp'] = (
        (
            df.loc[indices_latest_build_per_job, ['max_timestamp']] + dt.timedelta(hours=12)
        )
        .assign(now = pd.to_datetime("now"))
    ).max(axis=1)

    return df

# TODO: Remove if proven impossible after prior extrapolations:
# # Fixes the last sample of the sequence if we have made extrapolations
# def correct_extrapolations(df):
#     df = df.copy()

#     cond = (df.min_timestamp > df.max_timestamp)
#     df.loc[cond, 'max_timestamp'] = df.loc[cond, 'min_timestamp']

#     return df

# Main function
def create_sequence(df, grp_cols, agg_outcome, result_name):

    aggregations_dict = {'timestamp': ['min', 'max'], 'build': ['min', 'max']}
    aggregations_dict = {**aggregations_dict, **agg_outcome}

    return (
        df
        .groupby(by=grp_cols)
        .agg(aggregations_dict)
        .dropna()

        # Fixes column headers after complex `groupby`:
        .pipe(flatten_multi_level, result_name)

        .astype({'min_build': int, 'max_build': int})
        .reset_index()

        # Extends the length of each sequence up to the beginning of the next sequence:
        .pipe(extend_sequence, agg=grp_cols[:-1]) # We have already aggregated by the last 'grp_*' ID.

        # Extend the right edge of the last sample of each group to the end of the observed period
        .pipe(extend_last_sample_per_group, agg=grp_cols[:-1])

        # Extend the sequences from the last build of each job to have some extra width to be visible
        .pipe(extend_lastest_build_per_job)

        # TODO: Remove if proven impossible after prior extrapolations:
        # # If we have made extrapolations, the last sample of the sequence may also need re-adjustment at its end (`max_timestamp`)
        # .pipe(correct_extrapolations)

        # Add column with the duration of each period
        .assign(duration = lambda x: (x.max_timestamp - x.min_timestamp))
    )

In [ ]:
# Summarizes the conditions for the 3 types of sequences in different tables

# Was the build successful?: If at least one in the sequence is 'FAILURE', the whole sequence is in failure
agg_build_result = lambda x: 'FAILURE' if (x=='FAILURE').any() else 'SUCCESS'

# Were all Robot tests successful?: If at least one in the sequence is 'FAIL', the whole sequence is failing
agg_test_result = lambda x: 'FAIL' if (x=='FAIL').any() else 'PASS'

# Was all the building and testing successful?: If at least one in the sequence is 'PASS', the whole sequence is passing tests
agg_success_fail = lambda x: 'PASS' if (x=='PASS').any() else 'FAIL'

In [ ]:
sequence_build_result = create_sequence(data, grp_cols=['job', 'grp_build_result'], agg_outcome={'build_result': agg_build_result}, result_name='build_result')
sequence_test_result = create_sequence(data, grp_cols=['job', 'grp_test_result'], agg_outcome={'test_result': agg_test_result}, result_name='test_result')
sequence_success_fail = create_sequence(data, grp_cols=['job', 'grp_success_fail'], agg_outcome={'test_result': agg_success_fail}, result_name='success_fail')

In [ ]:
# Sometimes, Jenkins is able to create a test report, but it is unable to complete a proper build and image upload
# This function allows to discount this effect
def makes_stricter(df, change_to='UNAVAILABLE'):
    df = df.copy()

    cond = (df.build_result=='FAILURE') & (df.test_result=='PASS')
    df.loc[cond, 'test_result'] = change_to

    return df

sequence_test_result_strict = create_sequence(data.pipe(makes_stricter), grp_cols=['job', 'grp_test_result'], agg_outcome={'test_result': agg_test_result}, result_name='test_result')
sequence_success_fail_strict = create_sequence(data.pipe(makes_stricter, 'FAIL'), grp_cols=['job', 'grp_success_fail'], agg_outcome={'test_result': agg_success_fail}, result_name='success_fail')

In [ ]:
# #display(data)
# display(sequence_build_result.head(15))
# display(sequence_test_result.head(15))
# display(sequence_success_fail.head(15))

In [ ]:
print('DONE')

### 2.3 Finding sequences of pass/fails per test suite

In [ ]:
# Convenient alias to allow time masks

# test_suites_data = df_all_build_reports.query('(timestamp>"2021-08-01") & (timestamp<"2021-08-20")').copy().drop(columns=['source'])
test_suites_data = df_all_build_reports

if 'first_date' in locals():
    test_suites_data = test_suites_data.query('timestamp>=@first_date')

if 'last_date' in locals():
    # Needs to include latest hour of the last day
    last_timestamp = pd.Timestamp(last_date) + dt.timedelta(days=1)
    test_suites_data = test_suites_data.query('timestamp<@last_timestamp')

test_suites_data = test_suites_data.copy()


In [ ]:
# Adds columns with groups labels
test_suites_data = (
    test_suites_data
    .assign(
        grp_test_result = lambda x: find_sequence_number(x, relevant_col='status', mapping=mapping_test_result, grouping=['job', 'name']),
        )
)

In [ ]:
sequence_test_suites = create_sequence(test_suites_data, grp_cols=['job', 'name', 'grp_test_result'], agg_outcome={'status': agg_test_result}, result_name='test_result')

In [ ]:
#sequence_test_suites.info()

In [ ]:
# If requested, it also dumps all the calculated sequences

if dump_sequences:
    filename = os.path.join(outputs_folder, 'sequences_dump.xlsx')
    with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
        sequence_build_result.to_excel(writer, index=False, sheet_name='sequence_build_result')
        sequence_test_result.to_excel(writer, index=False, sheet_name='sequence_test_result')
        sequence_success_fail.to_excel(writer, index=False, sheet_name='sequence_success_fail')
        sequence_test_suites.to_excel(writer, index=False, sheet_name='sequence_test_suites')

In [ ]:
print('DONE')

## 3. Reports

In [ ]:
plt.style.use('fivethirtyeight')

### 3.1 Aggregated success rate per test step

In [ ]:
def plot_aggregated_success_rate(data_filtered, title, filename=None):

    fig, ax = plt.subplots(figsize = (12,6))

    t = data_filtered.timestamp
    pass_pct = 100 * data_filtered.pass_pct
    fail_pct = 100 * data_filtered.fail_pct
    #unavailable = (data_filtered.test_result=='UNAVAILABLE')*100
    unavailable = (
        data_filtered.test_result.map({'UNAVAILABLE': 100})
        .fillna(method='ffill', limit=1)
        .fillna(method='bfill', limit=1)
    )

    ax.fill_between(t, fail_pct+pass_pct, pass_pct, color='red', alpha=0.5, label='Failed')
    ax.fill_between(t, pass_pct, color='lime', alpha=0.5, label='Passed')
    ax.fill_between(t, unavailable, color='dimgray', label='Unsuccessful builds')
    ax.axhline(100, color='black', linewidth=2, linestyle='--')

    ax.set_title(title, fontsize=16)
    ax.legend(fontsize=12, fancybox=True, shadow=True, borderpad=1, bbox_to_anchor = (1, 1))
    #ax.legend(fontsize=12, fancybox=True, shadow=True, borderpad=1, bbox_to_anchor = (0.32, 0.4))
    fig.autofmt_xdate()

    fig.tight_layout()

    if (filename):
        fig.savefig(filename + '.png', dpi=300)
        fig.savefig(filename + '.svg')

    plt.show()

In [ ]:
file_names = ['fully_successful_builds_v10', 'fully_successful_builds_master', 'fully_successful_builds_v9']

for relevant_job, job_name, file_name in zip(relevant_jobs, job_names, file_names):
    display(
        _ = plot_aggregated_success_rate(
            data.query("job==@relevant_job"),
            f'{job_name} - % of successful test steps ({today})',
            os.path.join(outputs_folder, file_name)
        )
    )

### 3.2 Overall success of Jenkins builds and Robot tests

In [ ]:
def plot_aggregated_builds_and_tests(data_filtered, state_col, title, ok_states, nok_states, filename=None):

    fig, ax = plt.subplots(figsize = (12,6))

    for index, row in data_filtered.iterrows():
        #color = 'red' if row.build_result=='FAILURE' else 'lime'
        color = 'red' if row[state_col] in nok_states else 'lime'
        ax.axvspan(row.min_timestamp, row.max_timestamp, color=color, alpha=0.5)

    ax.set_title(title, fontsize=16)
    #ax.legend(fontsize=12, fancybox=True, shadow=True, borderpad=1, bbox_to_anchor = (1, 1))
    fig.autofmt_xdate()

    fig.tight_layout()

    if (filename):
        fig.savefig(filename + '.png', dpi=300)
        fig.savefig(filename + '.svg')

    plt.show()

In [ ]:
if extended_print:

    file_names = ['successful_failed_builds_v10', 'successful_failed_builds_master', 'successful_failed_builds_v9']

    for relevant_job, job_name, file_name in zip(relevant_jobs, job_names, file_names):
        display(
            _ = plot_aggregated_builds_and_tests(
                sequence_build_result.query("job==@relevant_job"),
                'build_result',
                f'{job_name} - Build completions and failures ({today})',
                ok_states=['SUCCESS'], nok_states=['FAILURE'],
                filename=os.path.join(outputs_folder, file_name)
                )
        )

In [ ]:
if extended_print:

    file_names = ['global_robot_status_v10', 'global_robot_status_master', 'global_robot_status_v9']

    for relevant_job, job_name, file_name in zip(relevant_jobs, job_names, file_names):
        display(
            _ = plot_aggregated_builds_and_tests(
                sequence_test_result.query("job==@relevant_job"),
                'test_result',
                f'{job_name} - Robot tests status ({today})',
                ok_states=['PASS'], nok_states=['FAIL'],
                filename=os.path.join(outputs_folder, file_name)
                )
        )

In [ ]:
if extended_print:

    file_names = ['global_stability_status_v10', 'global_stability_status_master', 'global_stability_status_v9']

    for relevant_job, job_name, file_name in zip(relevant_jobs, job_names, file_names):
        display(
            _ = plot_aggregated_builds_and_tests(
                sequence_success_fail.query("job==@relevant_job"),
                'success_fail',
                f'{job_name} - Stability for point release ({today})',
                ok_states=['PASS'], nok_states=['FAIL'],
                filename=os.path.join(outputs_folder, file_name)
                )
        )

In [ ]:
def plot_aggregated_stability_sequences(sequences, state_cols, titles, ok_states, nok_states, text=None, suptitle=None, filename=None, figsize=(14,8), tight=False):

    #fig, ax = plt.subplots(nrows=3, sharex=True, figsize = (14,8))
    fig, ax = plt.subplots(nrows=len(sequences), sharex=True, figsize=figsize)

    for i in range(len(sequences)):
        for index, row in sequences[i].iterrows():
            color = 'red' if row[state_cols[i]] in nok_states[i] else 'lime'
            ax[i].axvspan(row.min_timestamp, row.max_timestamp, color=color, alpha=0.5)
            if text:
                ax[i].text(0.5, 0.5, text[i], dict(size=14),
                           horizontalalignment='center', verticalalignment='center', transform=ax[i].transAxes, rasterized=False)

        if not text:
            ax[i].set_title(titles[i], fontsize=16)
        ax[i].set_yticklabels([])
        #ax[i].legend(fontsize=12, fancybox=True, shadow=True, borderpad=1, bbox_to_anchor = (1, 1))

    fig.autofmt_xdate()

    if tight:
        fig.tight_layout()
    if suptitle:
        fig.suptitle(suptitle, fontsize=22)

    if (filename):
        fig.savefig(filename + '.png', dpi=300)
        fig.savefig(filename + '.svg')

    plt.show()

In [ ]:
file_names = ['global_compared_stability_status_v10', 'global_compared_stability_status_master', 'global_compared_stability_status_v9']

for relevant_job, job_name, file_name in zip(relevant_jobs, job_names, file_names):
    display(
        _ = plot_aggregated_stability_sequences(
            sequences=[
                sequence_build_result.query("job==@relevant_job"),
                #sequence_test_result.query("job==@relevant_job"),
                sequence_test_result_strict.query("job==@relevant_job"),
                #sequence_success_fail.query("job==@relevant_job")
                sequence_success_fail_strict.query("job==@relevant_job")
            ],
            state_cols=['build_result', 'test_result', 'success_fail'],
            titles=[
                'Build completions and failures',
                'Robot tests status',
                'Stability for point release'
            ],
            suptitle=f'{job_name} - Robot tests status ({today})\n',
            ok_states=[['SUCCESS'], ['PASS'], ['PASS']],
            nok_states=[['FAILURE'], ['FAIL'], ['FAIL']],
            filename=os.path.join(outputs_folder, file_name)
        )
    )


In [ ]:
# display(sequence_build_result.query("job==@relevant_jobs[0]"))
# display(sequence_test_result.query("job==@relevant_jobs[0]"))
# display(sequence_success_fail.query("job==@relevant_jobs[0]"))

### 3.3 Sequences of pass/fails per test suites

In [ ]:
# Rearranges the sequence as a list of sequences per 'job' x 'suite'
def prepare_suite_sequences_for_plotting(df_suites):

    jobs = []
    suites = []
    sequences = []
    for name, group in df_suites.groupby(['job', 'name']):
        jobs.append(name[0])
        suites.append(name[1])
        sequences.append(group)

    return jobs, suites, sequences

In [ ]:
file_names = ['success_per_test_suite_status_v10', 'success_per_test_suite_status_master', 'success_per_test_suite_status_v9']

for relevant_job, job_name, file_name in zip(relevant_jobs, job_names, file_names):

    sequence_suites_filtered = sequence_test_suites.query('job==@relevant_job')
    _, suites, sequences = prepare_suite_sequences_for_plotting(sequence_suites_filtered)

    display(
        _ = plot_aggregated_stability_sequences(
            sequences=sequences,
            state_cols = ['test_result'] * len(sequences),
            titles = [''] * len(sequences),
            text = suites,
            suptitle = f'{job_name} ({today})',
            ok_states = ['PASS'] * len(sequences),
            nok_states = ['FAIL'] * len(sequences),
            filename = os.path.join(outputs_folder, file_name),
            figsize = (18,16),
            tight = False
        )
    )

### 3.4 Failing days per test suite

In [ ]:
def plot_days_suites_ok_nok(suites, days_failing, days_passing, title, filename=None):

    fig, ax = plt.subplots(figsize = (12,16))

    plt.barh(suites, days_failing, color='red', alpha=0.5, label='Failing')
    plt.barh(suites, days_passing, color='lime', alpha=0.5, left=days_failing, label='Passing')

    #ax.set_title(title, fontsize=20)
    fig.suptitle(title, fontsize=20)
    ax.set_xlabel('Number of days')
    #ax.legend(fontsize=12, fancybox=True, shadow=True, borderpad=1, bbox_to_anchor = (1, 1))

    fig.tight_layout()

    if (filename):
        fig.savefig(filename + '.png', dpi=300)
        fig.savefig(filename + '.svg')

    plt.show()

In [ ]:
def round_to_days(df):
    df = df.copy()

    df['FAIL'] = pd.to_timedelta(df.FAIL) / pd.Timedelta(days=1)
    df['PASS'] = pd.to_timedelta(df.PASS) / pd.Timedelta(days=1)

    # # Round to whole days
    # df['FAIL'] = df.FAIL.round(0).astype(int)
    # df['PASS'] = df.PASS.round(0).astype(int)

    return df

In [ ]:
file_names = ['failing_days_per_suite_v10', 'failing_days_per_suite_master', 'failing_days_per_suite_v9']

for relevant_job, job_name, file_name in zip(relevant_jobs, job_names, file_names):

    sequence_suites_filtered = sequence_test_suites.query('job==@relevant_job')

    fail_pass_durations_per_suite = (
        sequence_suites_filtered
        .pivot_table(
            index = 'name',
            columns = 'test_result',
            values = 'duration',
            aggfunc = 'sum'
        )
        .fillna(0)
        .reset_index()
        .sort_values(['FAIL', 'PASS'])
        .pipe(round_to_days)
    )

    display(
        _ = plot_days_suites_ok_nok(
            suites=fail_pass_durations_per_suite.name,
            days_failing=fail_pass_durations_per_suite.FAIL,
            days_passing=fail_pass_durations_per_suite.PASS,
            title=f'{job_name} - Failing days per test suite ({today})',
            filename=os.path.join(outputs_folder, file_name)
        )
    )

## 4. Information about the latest builds of relevant jobs

In [ ]:
# Uncomment to use external notebook
#%run 001-analysis_latest_build.ipynb

In [ ]:
def load_latest_builds_all_jobs(engine, too_old_builds='1980-12-15'):
	'''
	From each of the known jobs, retrieves their latest build.
	Returns a dataframe with a row per job.

	Usage:

	load_latest_builds(engine, too_old_builds='1980-12-15')

	- `engine`: Database engine to use for the connection.
	- `too_old_builds`: Limits the query to builds not older than a date. By default, it does not limit in practice (1980!).
	'''

	table_known_builds = 'builds_info'

	query_latest_builds = f'''
	SELECT main.*
	FROM {table_known_builds} AS main
	INNER JOIN (
		SELECT job, MAX(timestamp) as ts
		FROM {table_known_builds}
		WHERE timestamp>DATETIME("{too_old_builds}")
		GROUP BY job
	) AS latest_build
	ON main.job=latest_build.job AND main.timestamp=ts
	'''

	with engine.begin() as conn:
		df_latest_builds = pd.read_sql(query_latest_builds, con=conn)

	df_latest_builds['timestamp'] = pd.to_datetime(df_latest_builds.timestamp)

	return df_latest_builds

In [ ]:
def load_latest_report_all_jobs(engine, too_old_builds='1980-12-15'):
	'''
	From each of the known jobs, retrieves the report from their latest build.
	Returns a dataframe with a row per suite per job (in case the latest build of the job generated a report).

	Usage:

	load_latest_report_all_jobs(engine, too_old_builds='1980-12-15')

	- `engine`: Database engine to use for the connection.
	- `too_old_builds`: Limits the query to builds not older than a date. By default, it does not limit in practice (1980!).

	'''
	table =  'robot_reports'
	table_known_builds = 'builds_info'

	query_robot_reports = f'''
	SELECT details.*
	FROM {table} AS details
	INNER JOIN {table_known_builds} AS main
	ON details.job=main.job AND details.build=main.build
	INNER JOIN (
		SELECT job, MAX(timestamp) as ts
		FROM {table_known_builds}
		WHERE timestamp>DATETIME("{too_old_builds}")
		GROUP BY job
	) AS latest_build
	ON main.job=latest_build.job AND main.timestamp=ts
	'''

	with engine.begin() as conn:
		df_robot_reports = pd.read_sql(query_robot_reports, con=conn)

	df_robot_reports['starttime'] = pd.to_datetime(df_robot_reports.starttime)
	df_robot_reports['endtime'] = pd.to_datetime(df_robot_reports.endtime)

	return df_robot_reports

In [ ]:
def load_latest_extended_report_all_jobs(engine, too_old_builds='1980-12-15'):
	'''
	From each of the known jobs, retrieves the extended report from their latest build.
	Returns a dataframe with a row per test per suite per job (in case the latest build of the job generated a report).

	Usage:

	load_latest_extendend_report_all_jobs(engine, too_old_builds='1980-12-15')

	- `engine`: Database engine to use for the connection.
	- `too_old_builds`: Limits the query to builds not older than a date. By default, it does not limit in practice (1980!).
	'''

	table = 'robot_reports_extended'
	table_known_builds = 'builds_info'

	query_robot_reports = f'''
	SELECT details.*
	FROM {table} AS details
	INNER JOIN {table_known_builds} AS main
	ON details.job=main.job AND details.build=main.build
	INNER JOIN (
		SELECT job, MAX(timestamp) as ts
		FROM {table_known_builds}
		WHERE timestamp>DATETIME("{too_old_builds}")
		GROUP BY job
	) AS latest_build
	ON main.job=latest_build.job AND main.timestamp=ts
	'''

	with engine.begin() as conn:
		df_robot_reports_extended = pd.read_sql(query_robot_reports, con=conn)

	df_robot_reports_extended['starttime'] = pd.to_datetime(df_robot_reports_extended.starttime)
	df_robot_reports_extended['endtime'] = pd.to_datetime(df_robot_reports_extended.endtime)

	return df_robot_reports_extended

In [ ]:
engine = create_engine(database_uri)

df_latest_builds_all_jobs = load_latest_builds_all_jobs(engine, too_old_builds=too_old_builds)
df_latest_report_all_jobs = load_latest_report_all_jobs(engine, too_old_builds=too_old_builds)
df_latest_extended_report_all_jobs = load_latest_extended_report_all_jobs(engine, too_old_builds=too_old_builds)

### Latest build of each job

In [ ]:
display(df_latest_builds_all_jobs)

In [ ]:
for job_name, build in zip(df_latest_builds_all_jobs.job, df_latest_builds_all_jobs.build):
    stage, branch = job_name.split('/')
    link = f'https://osm.etsi.org/jenkins/view/Robot%20tests/job/{stage}/job/{branch}/{build}/'
    display(Markdown(f'[Click to see the details of **build {build} of {job_name}**]({link})'))

### Failed test suites per job (if any):

In [ ]:
df_failed = (
    df_latest_report_all_jobs
    .query("status=='FAIL'")
)

for job_name in relevant_jobs:
    stage, branch = job_name.split('/')
    link = f'https://osm.etsi.org/jenkins/view/Robot%20tests/job/{stage}/job/{branch}/{build}/robot/report/report.html'
    display(
        Markdown(f'**{job_name}:** ([full report]({link}))')
    )
    display(
        df_failed
        .query('job==@job_name')
        .drop(columns=['build', 'source', 'job', 'id', 'failed_test_id'])
    )

### Details of failed tests into failing test suites (if any):

In [ ]:
df_details_failed = (
    df_latest_extended_report_all_jobs
    .merge(
        df_failed[['job', 'build', 'name']],
        how='inner',
        left_on=['job', 'build', 'suite_name'],
        right_on=['job', 'build', 'name']
        )
    .drop(columns=['suite_id', 'test_id', 'name'])
    .query('status=="FAIL"')
)

for job_name in relevant_jobs:
    #display(job_name)
    stage, branch = job_name.split('/')
    link = f'https://osm.etsi.org/jenkins/view/Robot%20tests/job/{stage}/job/{branch}/{build}/robot/report/report.html'
    display(
        Markdown(f'**{job_name}:** ([full report]({link}))')
    )
    display(
        df_details_failed
        .query('job==@job_name')
        .drop(columns=['job', 'build'])
    )

In [ ]:
!jupyter nbconvert --to html --output report_outputs/analysis_of_test_results.html --TemplateExporter.exclude_input=True 01-analysis_of_test_results.ipynb

---

---


In [ ]:
### ANNEX: Samples of density plots (for future use)


In [ ]:
sequence_build_filtered = sequence_build_result[sequence_build_result.job=='osm-stage_3-merge/v9.0']
#sequence_build_filtered.iloc[0:5]

In [ ]:
def plot_density_builds_and_tests(data, title, labels, filename=None):

    fig, ax = plt.subplots(figsize = (12,6))

    n_bins = 25
    colors = ['green', 'red']
    #labels = ['OK', 'NOK']

    #ax.hist(data, n_bins, density=True, histtype='bar', color=colors, label=labels)
    #ax.hist(data, n_bins, density=True, histtype='bar', rwidth=0.8, label=labels, color=colors)
    ax.hist(data, n_bins, density=True, histtype='bar', rwidth=0.8, label=labels, color=colors, alpha=0.5)

    #ax.set_title(title, fontsize=20)
    fig.suptitle(title, fontsize=20)
    ax.set_xlabel('Duration of sequences (days)')
    ax.set_ylabel('Density')
    ax.legend(fontsize=12, fancybox=True, shadow=True, borderpad=1, bbox_to_anchor = (1, 1))

    fig.tight_layout()

    if (filename):
        fig.savefig(filename + '.png', dpi=300)
        fig.savefig(filename + '.svg')

    plt.show()

In [ ]:
build_durations_filtered = sequence_build_filtered.copy()
build_durations_filtered['duration'] = build_durations_filtered.max_timestamp - build_durations_filtered.min_timestamp
#build_durations_filtered['duration'] = build_durations_filtered['duration'] / pd.to_timedelta(1, unit='D') # Expressed in days, allowing decimals
build_durations_filtered['duration'] = build_durations_filtered['duration'].dt.days

build_durations_filtered.drop(columns=['grp_build_result', 'min_timestamp', 'max_timestamp', 'min_build', 'max_build'], inplace=True)

builds_ok = build_durations_filtered.loc[build_durations_filtered.build_result=='SUCCESS', 'duration']
builds_nok = build_durations_filtered.loc[build_durations_filtered.build_result=='FAILURE', 'duration']

In [ ]:
# plot_density_builds_and_tests(data=[builds_ok, builds_nok],
#                               title='Rel NINE branch - Histogram of durations of build states (failure/success) ('+ today + ')',
#                               labels=['SUCCESS', 'FAILURE'],
#                               filename=os.path.join(outputs_folder, 'density_build_success_failure_v9'))

In [ ]:
def plot_kde_builds_and_tests(data, x, hue, colors, title, filename=None):

    fig, ax = plt.subplots(figsize = (12,6))

    sns.kdeplot(data=data,
                x=x, hue=hue,
                cut=0, bw_adjust=0.2,
                palette=colors, fill=True, alpha=0.4,
                ax=ax)

    fig.suptitle(title, fontsize=20)
    ax.set_xlabel('Duration of sequences (days)')

    if filename:
        fig.savefig(filename + '.png', dpi=300)
        fig.savefig(filename + '.svg')

    plt.show()

In [ ]:
# plot_kde_builds_and_tests(data=build_durations_filtered,
#                           x='duration', hue='build_result', colors=['red', 'green'],
#                           title='Rel NINE branch - KDE of durations of build states (failure/success) ('+ today + ')',
#                           filename=os.path.join(outputs_folder, 'kde_build_success_failure_v9'))